In [1]:
# What questions do we want to answer with this model?
# What events do we want to simulate?
# Can we simulate the health of the community?

In [2]:
import random # psuedo randomness generator
import numpy as np # import numerical python
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

#imported some addition utilities to help with configuration set-up
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import append_configs
from cadCAD import configs

In [3]:
# What does this do?
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [4]:
# Params

initial_conditions = {
    'village_folk': {i:{'stuff': np.random.randint(1,10), 'tokens': 0} for i in range(10)}, # initialize village folk and their stuff and cred_castle balances
    'castle': 0, # the cred castle
    'castle_token_supply': 0, # the amount of castle tokens
}

In [5]:
initial_conditions

{'village_folk': {0: {'stuff': 4, 'tokens': 0},
  1: {'stuff': 3, 'tokens': 0},
  2: {'stuff': 8, 'tokens': 0},
  3: {'stuff': 3, 'tokens': 0},
  4: {'stuff': 6, 'tokens': 0},
  5: {'stuff': 8, 'tokens': 0},
  6: {'stuff': 5, 'tokens': 0},
  7: {'stuff': 6, 'tokens': 0},
  8: {'stuff': 5, 'tokens': 0},
  9: {'stuff': 6, 'tokens': 0}},
 'castle': 0,
 'castle_token_supply': 0}

In [6]:
# Everyone in CredLand is crazy 
def villagers_be_crazy(params, step, sL, s):
    print('go crazy')
    
    n = len(s['village_folk']) #get the length of the list of villager indices
    index = np.random.randint(0, n) #pick a random integer from (0,.., n-1)
    p = np.random.randint(10, 90)
    
    if (p <= 50):
        action = 'buy_castle_token'
        #i need to buy FROM someone
        #select the set of village_folk who have tokens
        holders = [i for i in range(n) if s['village_folk'][i]['tokens']>0]
        if len(holders)>0:
            friend = np.random.choice(holders)
            print('friend found')
            print(friend)
        else:
            action = None
            friend = None

    #you cannot sell tokens you don't have
    elif (p > 80)&(s['village_folk'][index]['tokens']>0):
        action = 'sell_castle_token'
        #i need to sell TO someone
        #select the set of village_folk who have stuff
        stuff_havers = [i for i in range(n) if s['village_folk'][i]['stuff']>0]
        friend = np.random.choice(stuff_havers)
    
    elif (p > 50):
        action = 'contribute'
        friend = None
    
    print('')
    print('State of things: ', s)
    return ({'action': action, 'villager': index, 'friend':friend})

In [ ]:
# Go to market
  # a policy function for villagers to decide if they want to buy, sell, or hold their castle tokens. 

In [ ]:
# Decide to build
    # a policy function for villagers to decide if they want to build the castle or not. 

In [7]:
# How villagers build the castle 
def build_castle(params, step, sL, s, _input):
    
    action = _input['action']
    y = 'castle' #this function updates the castle state
    x = s[y]
    if action == 'contribute':
        print('build_castle')
        x += 1
    return (y, x)

In [8]:
def account_castle_tokens(params, step, sL, s, _input):
    
    action = _input['action']
    y = 'castle_token_supply' #this function updates the castle state
    x = s[y]
    if action == 'contribute':
        print('account_castle_tokens')
        x += 1
    return (y, x)

In [9]:
# How villagers build the castle 
def royal_decree(params, step, sL, s, _input):
    
    action = _input['action']
    y = 'village_folk' #this function updates the castle state
    x = s[y]
    villager_index = _input['villager']
    if action == 'contribute':
        print('royal_decree')
        x[villager_index]['tokens'] = x[villager_index]['tokens'] + 1
    
    return (y, x)

In [10]:
# How villagers sell castle tokens
def sell_castle_tokens(params, step, sL, s, _input):
    
    action = _input['action']
    y = 'village_folk'
    x = s[y]
    
    if action == 'sell_castle_token':
        print('sell_castle_tokens')
        #villager_index = _input['person'] #you used the key "villager" not 'person' 
        villager_index = _input['villager']
        friend_index = _input['friend']
        #token accounting
        x[villager_index]['tokens'] -= 1 
        x[friend_index]['tokens'] += 1 #typo index <-- indes
        #stuff accounting
        x[villager_index]['stuff'] += 1 
        x[friend_index]['stuff'] -= 1 #typo index <-- indes

    return(y, x)

In [11]:
# How villagers buy castle tokens 
def buy_castle_tokens(params, step, sL, s, _input):
    
    action = _input['action']
    y = 'village_folk'
    x = s[y]

    if action == 'buy_castle_token':
        print('buy_castle_tokens')
        #villager_index = _input['person'] #you used the key "villager" not 'person' 
        villager_index = _input['villager']
        friend_index = _input['friend']
        #token accounting
        x[villager_index]['tokens'] += 1 
        x[friend_index]['tokens'] -= 1 #typo index <-- indes
        #stuff accounting
        x[villager_index]['stuff'] -= 1 
        x[friend_index]['stuff'] += 1 #typo index <-- indes
    
    return(y, x)

#note if you are copy pasting code, it probably means you could collapse it into one function
#eg, buy and sell here are the same thing but with a negative number for sell

In [12]:
# Remove indebted villagers 
def feed_to_dragons(params, step, sL, s, _input):
    
    y = 'village_folk'
    x = s[y]
    
    keys = x.keys()
    for k in keys: # iterate through all the villagers
        if any(v < 0 for v in k): # if any of the villager's assets are less than 0
            print('Shame, shame... ', villager) # shame the villager
            #x[k].remove(villager) # let the villager fend for themselves
    
    return(y, x)

In [13]:
# Everyone in CredLand is crazy 
partial_state_update_blocks = [
    { 
        'policies': {
            'villagers_be_crazy': villagers_be_crazy
        },
        'variables': {
            'castle': build_castle,
            'village_folk': royal_decree,
            'castle_token_supply': account_castle_tokens,
        }
    },
    {
        'policies': {
            'villagers_be_crazy': villagers_be_crazy
        },
        'variables': {
            'village_folk': buy_castle_tokens,
            'village_folk': sell_castle_tokens
        }
    },
    {
        'policies': {    
        },
        'variables': {
            'village_folk': feed_to_dragons
        }
    }
]

In [14]:
len(partial_state_update_blocks)

3

In [15]:
simulation_parameters = {
    'T': range(100),
    'N': 1,
    'M': {}
}

In [16]:
# i replaced this with another cell which uses "append_configs" utility instead 
#---
# config = Configuration(initial_state=initial_conditions,
#                        partial_state_update_blocks=partial_state_update_blocks,
#                        sim_config=simulation_parameters
#                       )


In [17]:
append_configs(
    initial_state=initial_conditions, #dict containing variable names and initial values
    partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
    sim_configs=simulation_parameters #dict containing simulation parameters
)

[{'T': range(0, 100), 'N': 1, 'M': {}}]


In [18]:
# What does this do and why doesn't it work?
# this provides instructions to your computer on how to run your model
# since nothing actually gets executed until this cell, errors the model above
# trigger errors when you run this cell
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, configs)
raw_result, tensor = executor.execute()


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: single_proc: [<cadCAD.configuration.Configuration object at 0x7f5e78c44a60>]
Configurations: [<cadCAD.configuration.Configuration object at 0x7f5e78c44a60>]
go crazy

State of things:  {'village_folk': {0: {'stuff': 4, 'tokens': 0}, 1: {'stuff': 3, 'tokens': 0}, 2: {'stuff': 8, 'tokens': 0}, 3: {'stuff': 3, 'tokens': 0}, 4: {'stuff': 6, 'tokens': 0}, 5: {'stuff': 8, 'tokens': 0}, 6: {'stuff': 5, 'tokens': 0}, 7: {'stuff': 6, 'tokens': 0}, 8: {'stuff': 5, 'tokens': 0}, 9: {'stuff': 6, 'tokens': 0}}, 'castle': 0, 'castle_token_supply': 0, 'run': 1, 'substep': 0, 'timestep': 0}
build_castle
royal_decree
account_castle_tokens


KeyError: 'action'

In [ ]:
import pandas as pd
df = pd.DataFrame(raw_result)

In [ ]:
df.head()